In [1]:
import numpy as np
import pandas as pd
import matplotlib
import matplotlib.pyplot as plt
%matplotlib inline

In [16]:
data = pd.read_csv('spotify.csv')
data.drop('Unnamed: 0',axis=1,inplace=True)
data = data.drop_duplicates(subset = 'track_id')
genres = pd.read_csv('artist_genre.csv')
genres = genres.set_index('artist_id')

In [3]:
cols = ['acousticness','danceability','energy','instrumentalness','liveness',\
        'loudness','popularity','speechiness','tempo','track_id']
X = data[cols]
X = X.set_index('track_id')
X = X.drop_duplicates()
X = (X - X.min()) / (X.max() - X.min())

In [98]:
def make_playlist(df,track_id,genres,num_songs):
    """
    Function to make playlist from seed song.
    
    """
    cols = ['acousticness','danceability','energy','instrumentalness','liveness',\
        'loudness','popularity','speechiness','tempo','track_id']
    X = df[cols]
    X = X.set_index('track_id')
    X = (X - X.min()) / (X.max() - X.min())
    
    # get distance to all songs 
    song = X.loc[track_id].values
    vals = X.values 
    dists = np.linalg.norm(vals - song,axis = 1)
    sort_ind = np.argsort(dists)
    
    # create playlist by choosing closest tracks in same genre
    playlist = []
    ii = 0
    artist = df[df['track_id'].isin([track_id])]['artist_id'].values[0]
    song_genres = genres.loc[artist].index[genres.loc[artist].notnull()]
    
    for ii in sort_ind:
        if len(playlist) == num_songs:
            continue 
        new_song = X.index[ii]
        new_artist = df[df['track_id'].isin([new_song])]['artist_id'].values[0]
        new_genres = genres.loc[new_artist].index[genres.loc[new_artist].notnull()]
        genre_inter = np.intersect1d(song_genres,new_genres)
        if len(genre_inter) > 0:
            playlist.append(ii)
            
    return X.index[playlist].values

In [120]:
songs = ['14XWXWv5FoCbFzLksawpEe','0xMd5bcWTbyXS7wPrBtZA6','5YTM7OgQDt9EJAYcfHLniJ','4P6n0udLAnmCNXVcobYbsK']
for song in songs:
    playlist = make_playlist(data,song,genres,20)
    print(': '.join(data[data['track_id'].isin([song])][['artist','track_name']].values[0]))
    print(data[data['track_id'].isin(playlist)][['artist','track_name']],'\n')

The Doors: Riders On The Storm
                      artist                                track_name
577               Paul Simon       Me and Julio Down by the Schoolyard
2190         Tears For Fears         Everybody Wants To Rule The World
3184                    Cake                            I Will Survive
8776            Sonny & Cher        I Got You Babe - LP/Single Version
18546            OneRepublic                                 Apologize
19881         Lynyrd Skynyrd                        Sweet Home Alabama
21565                Nirvana                          Heart-Shaped Box
21683              The Doors      Break On Through (To The Other Side)
21690              The Doors                       Riders On The Storm
22682            The Beatles                        Help! - Remastered
23515            OneRepublic  If I Lose Myself - Alesso vs OneRepublic
25654                  Heart        All I Wanna Do Is Make Love To You
32310                  Cream                  